In [ ]:
def spikes(y,vlambda,vrho):

    import numpy as np
    import cvxpy as cp
    import scipy as scipy
    import cvxopt as cvxopt

    n = y.size

    # Form second difference matrix.
    e = np.ones((1, n))
    D = scipy.sparse.spdiags(np.vstack((e, -2*e, e)), range(3), n-2, n)

    # Solve l1 trend filtering problem + spikes.
    x = cp.Variable(shape=n)
    u = cp.Variable(shape=n)
    obj = cp.Minimize(0.5 * cp.sum_squares(y - x - u)
                      + vlambda * cp.norm(D*x, 1)
                      + vrho * cp.norm(u, 1) )

    prob = cp.Problem(obj)

    # ECOS and SCS solvers fail to converge before
    # the iteration limit. Use CVXOPT instead.
    prob.solve(solver=cp.CVXOPT, verbose=True)
    print('Solver status: {}'.format(prob.status))

    # Check for error.
    if prob.status != cp.OPTIMAL:
        raise Exception("Solver did not converge!")

    print("optimal objective value: {}".format(obj.value))

    return np.array(x.value), np.array(u.value)

In [ ]:
def level(y,vlambda,vrho):

    import numpy as np
    import cvxpy as cp
    import scipy as scipy
    import cvxopt as cvxopt

    n = y.size

    # Form second difference matrix.
    e = np.ones((1, n))
    D = scipy.sparse.spdiags(np.vstack((e, -2*e, e)), range(3), n-2, n)

    # Form the first difference matrix
    #e = np.zeros((1, n))
    #e[0,1:] = 1
    D1 = scipy.sparse.spdiags(np.vstack((-e, e)), range(2), n-1, n)

    # Solve l1 trend filtering problem + levels.
    x = cp.Variable(shape=n)
    u = cp.Variable(shape=n)
    obj = cp.Minimize(0.5 * cp.sum_squares(y - x - u)
                      + vlambda * cp.norm(D @ x, 1)
                      + vrho * cp.norm(D1 @  u, 1)
                      + vrho/10000 * cp.sum_squares( u ) )

    prob = cp.Problem(obj)

    # ECOS and SCS solvers fail to converge before
    # the iteration limit. Use CVXOPT instead.
    prob.solve(solver=cp.CVXOPT, verbose=True)
    print('Solver status: {}'.format(prob.status))

    # Check for error.
    if prob.status != cp.OPTIMAL:
        raise Exception("Solver did not converge!")

    print("optimal objective value: {}".format(obj.value))

    return np.array(x.value), np.array(u.value)



In [ ]:
from pyacs.gts.Sgts import Sgts
import matplotlib.pyplot as plt
%matplotlib qt

ts_dir = '/Users/nocquet/Dropbox/sse_project/ecuador/pck'
ts = Sgts(ts_dir, verbose=False)
#ts.CABP.plot()
mts = ts.CABP.extract_periods([2015.5,2016.5]).data[:,2]
f,o = l1filter(y,1,.01)
print(y.shape)
plt.plot(y);plt.plot(o);plt.plot(f)

In [ ]:
#!usr/env/bin python
# author: WU Dingcheng
import cvxpy as cvx
import numpy as np
from scipy.sparse import eye, csr_matrix, hstack, linalg
from scipy.signal import argrelextrema
from .fit import TSFit
from ..models.offsets import OffsetEvent, TrendChangeEvent


def gen_d2(n):
    """
    Generate the 2nd difference matrix.
    :param n: int, length of time series
    :return: csr_matrix, sparse matrix
    """
    I2 = eye(n - 2)
    O2 = csr_matrix((n - 2, 1))
    return hstack((I2, O2, O2)) + hstack((O2, -2 * I2, O2)) + hstack((O2, O2, I2))


def gen_d1(n):
    """
    Generate the 1st difference matrix
    :param n: int, length of time series
    :return: csr_matrix, sparse matrix
    """
    I1 = eye(n - 1)
    O1 = csr_matrix((n - 1, 1))
    return hstack((I1, O1)) - hstack((O1, I1))


def get_max_lam(y):
    """
    Calculate the max lambda value for given time series y
    :param y: np.array, time series given
    :return: float, max lambda value
    """
    D = gen_d2(len(y))
    ddt = D.dot(D.T)
    dy = D.dot(y)
    return np.linalg.norm(linalg.spsolve(ddt, dy), np.inf)


def get_max_rho(y):
    """
    Calculate the max rho value for given time series y,
    :param y: np.array, time series given
    :return: float, max rho value
    """
    D = gen_d1(len(y))
    ddt = D.dot(D.T)
    dy = D.dot(y)
    return np.linalg.norm(linalg.spsolve(ddt, dy), np.inf)


def l1filter(t, y,
             lam=1200,
             rho=80,
             periods=(365.25, 182.625),
             solver=cvx.MOSEK,
             verbose=False):
    """
    Do l1 regularize for given time series.
    :param t: np.array, time
    :param y: np.array, time series value
    :param lam: lambda value
    :param rho: rho value
    :param periods: list, periods, same unit as t
    :param solver: cvx.solver
    :param verbose: bool, show verbose or not
    :return: x, w, s, if periods is not None, else return x, w
    """
    t = np.asarray(t, dtype=np.float64)
    t = t - t[0]
    y = np.asarray(y, dtype=np.float64)

    assert y.shape == t.shape

    n = len(t)
    D = gen_d2(n)

    x = cvx.Variable(n)
    w = cvx.Variable(n)
    errs = y - x - w
    seasonal = None
    if periods:
        tpi_t = 2 * np.pi * t
        for period in periods:
            a = cvx.Variable()
            b = cvx.Variable()
            temp = a * np.sin(tpi_t / period) + b * np.cos(tpi_t / period)
            if seasonal is None:
                seasonal = temp
            else:
                seasonal += temp
        errs = errs - seasonal
    obj = cvx.Minimize(0.5 * cvx.sum_squares(errs) +
                       lam * cvx.norm(D * x, 1) +
                       rho * cvx.tv(w))
    prob = cvx.Problem(obj)
    prob.solve(solver=solver, verbose=verbose)
    if periods:
        return np.array(x.value), np.array(w.value), np.array(seasonal.value)
    else:
        return np.array(x.value), np.array(w.value), None
    t = np.asarray(t, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)
    n = len(t)
    x = cvx.Variable(n)
    w = cvx.Variable(n)
    dx = cvx.mul_elemwise(1.0 / np.diff(t), cvx.diff(x))
    x_term = cvx.tv(dx)
    dw = cvx.mul_elemwise(1.0 / np.diff(t), cvx.diff(w))
    w_term = cvx.norm(dw, 1)
    errs = y - x - w
    seasonal = None
    if periods:
        tpi_t = 2 * np.pi * t
        for period in periods:
            a = cvx.Variable()
            b = cvx.Variable()
            temp = a * np.sin(tpi_t / period) + b * np.cos(tpi_t / period)
            if seasonal is None:
                seasonal = temp
            else:
                seasonal += temp
        errs = errs - seasonal

    obj = cvx.Minimize(0.5 * cvx.sum_squares(errs) + lam * x_term + rho * w_term)
    prob = cvx.Problem(obj)
    prob.solve(solver=solver, verbose=verbose)
    if periods:
        return np.array(x.value), np.array(w.value), np.array(seasonal.value)
    else:
        return np.array(x.value), np.array(w.value), None


def detrend(t, y):
    pass


def l1wrapper(series,
              name,
              solver=cvx.MOSEK,
              periods=(0.5, 1),
              polys=1,
              lam=1200,
              rho=80,
              threshold=200,
              offset=True,
              trendchange=True):
    if not (offset or trendchange):
        return []
    x, w, s = l1filter(series['t'], series['y'], periods=periods, lam=lam, rho=rho, solver=solver)

    discontinuities = []
    d2x = np.abs(np.diff(x, 2))
    d2x[d2x < 0.01] = 0
    # d2x[d2x < np.percentile(d2x, 95)] = 0.0
    ind = argrelextrema(d2x, np.greater, order=7)
    flag = ind[0] + 2
    trendchanges = series.index[flag]
    for t in trendchanges:
        temp = TrendChangeEvent(t, name)
        discontinuities.append(temp)

    dw = np.abs(np.diff(w))
    dw[dw < 0.1] = 0.0
    # dw[dw < np.percentile(dw, 95)] = 0.0
    ind = argrelextrema(dw, np.greater, order=7)
    flag = ind[0] + 1
    offsets = series.index[flag]
    for o in offsets:
        temp = OffsetEvent(o, name)
        discontinuities.append(temp)

    if len(discontinuities) == 0:
        return discontinuities

    tsfit = TSFit(series)
    discontinuities = tsfit.discontinuitiesSignificanceTest(discontinuities, polys=polys, periods=periods)
    # return discontinuities
    # trendchanges = [i for i in discontinuities if isinstance(i, TrendChangeEvent)]
    # offsets = [i for i in discontinuities if isinstance(i, OffsetEvent)]
    discontinuities = tsfit.discontinutyFTest(F=threshold,
                                              polys=polys,
                                              periods=periods,
                                              discontinuities=discontinuities)
    # for discontinuity in discontinuities:
    #     temp = deepcopy(discontinuities)
    #     flag = tsfit.discontinutyFTest(discontinuity,
    #                                    F=threshold,
    #                                    polys=polys,
    #                                    periods=periods,
    #                                    discontinuities=temp)
    #     if not flag:
    #         discontinuities.remove(discontinuity)

    # for discontinuity in discontinuities:
    #     temp = deepcopy(discontinuities)
    #     flag = tsfit.discontinutyFTest(discontinuity,
    #                                    F=threshold,
    #                                    polys=polys,
    #                                    periods=periods,
    #                                    discontinuities=temp)
    #     if not flag:
    #         discontinuities.remove(discontinuity)
    temp = []
    for discontinuity in discontinuities:
        if offset and isinstance(discontinuity, OffsetEvent):
            temp.append(discontinuity)
        if trendchange and isinstance(discontinuity, TrendChangeEvent):
            temp.append(discontinuity)
    return temp

In [54]:
import cvxpy as cvx
import numpy as np
from scipy.sparse import eye, csr_matrix, hstack, linalg
from scipy.signal import argrelextrema

def gen_d2(n):
    """
    Generate the 2nd difference matrix.
    :param n: int, length of time series
    :return: csr_matrix, sparse matrix
    """
    I2 = eye(n - 2)
    O2 = csr_matrix((n - 2, 1))
    return hstack((I2, O2, O2)) + hstack((O2, -2 * I2, O2)) + hstack((O2, O2, I2))


def gen_d1(n):
    """
    Generate the 1st difference matrix
    :param n: int, length of time series
    :return: csr_matrix, sparse matrix
    """
    I1 = eye(n - 1)
    O1 = csr_matrix((n - 1, 1))
    return hstack((I1, O1)) - hstack((O1, I1))


def get_max_lam(y):
    """
    Calculate the max lambda value for given time series y
    :param y: np.array, time series given
    :return: float, max lambda value
    """
    D = gen_d2(len(y))
    ddt = D.dot(D.T)
    dy = D.dot(y)
    return np.linalg.norm(linalg.spsolve(ddt, dy), np.inf)


def get_max_rho(y):
    """
    Calculate the max rho value for given time series y,
    :param y: np.array, time series given
    :return: float, max rho value
    """
    D = gen_d1(len(y))
    ddt = D.dot(D.T)
    dy = D.dot(y)
    return np.linalg.norm(linalg.spsolve(ddt, dy), np.inf)


def l1filter(t, y,
             lam=1200,
             rho=80,
             periods=(365.25, 182.625),
             solver=cvx.MOSEK,
             verbose=False):
    """
    Do l1 regularize for given time series.
    :param t: np.array, time
    :param y: np.array, time series value
    :param lam: lambda value
    :param rho: rho value
    :param periods: list, periods, same unit as t
    :param solver: cvx.solver
    :param verbose: bool, show verbose or not
    :return: x, w, s, if periods is not None, else return x, w
    """

    t = np.asarray(t, dtype=np.float64)
    t = t - t[0]
    y = np.asarray(y, dtype=np.float64)

    assert y.shape == t.shape

    n = len(t)
    D = gen_d2(n)

    x = cvx.Variable(n)
    w = cvx.Variable(n)
    errs = y - x - w
    seasonal = None
    if periods:
        tpi_t = 2 * np.pi * t
        for period in periods:
            a = cvx.Variable()
            b = cvx.Variable()
            temp = a * np.sin(tpi_t / period) + b * np.cos(tpi_t / period)
            if seasonal is None:
                seasonal = temp
            else:
                seasonal += temp
        errs = errs - seasonal
    obj = cvx.Minimize(0.5 * cvx.sum_squares(errs) +
                       lam * cvx.norm(D * x, 1) +
                       rho * cvx.tv(w))
    prob = cvx.Problem(obj)
    prob.solve(solver=solver, verbose=verbose)
    if periods:
        return np.array(x.value), np.array(w.value), np.array(seasonal.value)
    else:
        return np.array(x.value), np.array(w.value), None
    t = np.asarray(t, dtype=np.float64)
    y = np.asarray(y, dtype=np.float64)
    n = len(t)
    x = cvx.Variable(n)
    w = cvx.Variable(n)
    dx = cvx.mul_elemwise(1.0 / np.diff(t), cvx.diff(x))
    x_term = cvx.tv(dx)
    dw = cvx.mul_elemwise(1.0 / np.diff(t), cvx.diff(w))
    w_term = cvx.norm(dw, 1)
    errs = y - x - w
    seasonal = None
    if periods:
        tpi_t = 2 * np.pi * t
        for period in periods:
            a = cvx.Variable()
            b = cvx.Variable()
            temp = a * np.sin(tpi_t / period) + b * np.cos(tpi_t / period)
            if seasonal is None:
                seasonal = temp
            else:
                seasonal += temp
        errs = errs - seasonal

    obj = cvx.Minimize(0.5 * cvx.sum_squares(errs) + lam * x_term + rho * w_term)
    prob = cvx.Problem(obj)
    prob.solve(solver=solver, verbose=verbose)
    if periods:
        return np.array(x.value), np.array(w.value), np.array(seasonal.value)
    else:
        return np.array(x.value), np.array(w.value), None


In [56]:
from pyacs.gts.Sgts import Sgts
import matplotlib.pyplot as plt
%matplotlib qt

ts_dir = '/Users/nocquet/Dropbox/sse_project/ecuador/pck'
ts = Sgts(ts_dir, verbose=False)
#ts.CABP.plot()
mts = ts.CABP.extract_periods([2015.5,2016.5])
t = mts.data[:,0]
y = mts.data[:,2]
f,o = l1filter(y,t)
print(y.shape)
plt.plot(y);plt.plot(o);plt.plot(f)


/usr/local/geodesy/anaconda38/lib/python3.8/site-packages/cvxpy/expressions/expression.py:516: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.

  warnings.warn(__STAR_MATMUL_WARNING__, UserWarning)


SolverError: The solver MOSEK is not installed.

In [ ]:
y.shape == t.shape
